In [1]:
pip install scikit-fuzzy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 24.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl

# Define fuzzy variables
temperature = ctrl.Antecedent(np.arange(0, 46, 1), 'temperature')
humidity = ctrl.Antecedent(np.arange(0, 101, 1), 'humidity')
wind_speed = ctrl.Antecedent(np.arange(0, 51, 1), 'wind_speed')

rain = ctrl.Consequent(np.arange(0, 101, 1), 'rain')
storm = ctrl.Consequent(np.arange(0, 101, 1), 'storm')
sunshine = ctrl.Consequent(np.arange(0, 101, 1), 'sunshine')

# Membership functions for inputs
temperature['low'] = fuzz.trapmf(temperature.universe, [0, 0, 10, 20])
temperature['medium'] = fuzz.trimf(temperature.universe, [10, 25, 35])
temperature['high'] = fuzz.trapmf(temperature.universe, [30, 40, 45, 45])

humidity['low'] = fuzz.trapmf(humidity.universe, [0, 0, 30, 50])
humidity['medium'] = fuzz.trimf(humidity.universe, [30, 60, 80])
humidity['high'] = fuzz.trapmf(humidity.universe, [70, 90, 100, 100])

wind_speed['low'] = fuzz.trapmf(wind_speed.universe, [0, 0, 10, 20])
wind_speed['medium'] = fuzz.trimf(wind_speed.universe, [10, 25, 35])
wind_speed['high'] = fuzz.trapmf(wind_speed.universe, [30, 40, 50, 50])

# Membership functions for outputs
rain['low'] = fuzz.trapmf(rain.universe, [0, 0, 25, 50])
rain['medium'] = fuzz.trimf(rain.universe, [25, 50, 75])
rain['high'] = fuzz.trapmf(rain.universe, [50, 75, 100, 100])

storm['low'] = fuzz.trapmf(storm.universe, [0, 0, 15, 40])
storm['medium'] = fuzz.trimf(storm.universe, [15, 50, 75])
storm['high'] = fuzz.trapmf(storm.universe, [50, 75, 100, 100])

sunshine['low'] = fuzz.trapmf(sunshine.universe, [0, 0, 25, 50])
sunshine['medium'] = fuzz.trimf(sunshine.universe, [25, 50, 75])
sunshine['high'] = fuzz.trapmf(sunshine.universe, [50, 75, 100, 100])

# Define fuzzy rules
rules = [
    ctrl.Rule(humidity['high'] & wind_speed['low'], rain['high']),
    ctrl.Rule(temperature['medium'] & humidity['medium'], rain['medium']),
    ctrl.Rule(wind_speed['high'] & humidity['high'], storm['high']),
    ctrl.Rule(humidity['medium'] & wind_speed['medium'], storm['medium']),
    ctrl.Rule(temperature['high'] & humidity['low'], sunshine['high']),
    ctrl.Rule(humidity['low'] & wind_speed['low'], sunshine['medium']),
    ctrl.Rule(temperature['low'] & humidity['low'], rain['low']),
    ctrl.Rule(temperature['high'] & wind_speed['medium'], sunshine['medium']),
    ctrl.Rule(temperature['medium'] & wind_speed['low'], sunshine['medium']),
]

# Create control system
weather_ctrl = ctrl.ControlSystem(rules)
weather_simulation = ctrl.ControlSystemSimulation(weather_ctrl)

# Main Program (Manual Input)
print("Welcome to the AI-Driven Weather Prediction System!")
print("Please enter the weather data manually:")

try:
    # Input values with validation
    temperature_input = float(input("Enter Temperature (°C) [0-45]: "))
    humidity_input = float(input("Enter Humidity (%) [0-100]: "))
    wind_speed_input = float(input("Enter Wind Speed (km/h) [0-50]: "))

    if not (0 <= temperature_input <= 45):
        raise ValueError("Temperature must be between 0 and 45°C.")
    if not (0 <= humidity_input <= 100):
        raise ValueError("Humidity must be between 0 and 100%.")
    if not (0 <= wind_speed_input <= 50):
        raise ValueError("Wind Speed must be between 0 and 50 km/h.")

    # Use manual data for fuzzy computation
    weather_simulation.input['temperature'] = temperature_input
    weather_simulation.input['humidity'] = humidity_input
    weather_simulation.input['wind_speed'] = wind_speed_input

    # Perform computation
    weather_simulation.compute()

    # Safely fetch results
    rain_result = weather_simulation.output.get('rain', 0)
    storm_result = weather_simulation.output.get('storm', 0)
    sunshine_result = weather_simulation.output.get('sunshine', 0)

    # Determine dominant weather condition
    dominant_condition = max(rain_result, storm_result, sunshine_result)
    if dominant_condition == rain_result:
        weather = "Rain"
    elif dominant_condition == storm_result:
        weather = "Storm"
    else:
        weather = "Sunshine"

    # Display results
    print("\nWeather Prediction Results:")
    print(f"Rain Probability: {rain_result:.2f}%")
    print(f"Storm Probability: {storm_result:.2f}%")
    print(f"Sunshine Probability: {sunshine_result:.2f}%")
    print(f"The predicted weather is: {weather}")

except ValueError as e:
    print(f"Input Error: {e}")

Welcome to the AI-Driven Weather Prediction System!
Please enter the weather data manually:
Enter Temperature (°C) [0-45]: 25
Enter Humidity (%) [0-100]: 60
Enter Wind Speed (km/h) [0-50]: 12

Weather Prediction Results:
Rain Probability: 50.00%
Storm Probability: 45.33%
Sunshine Probability: 50.00%
The predicted weather is: Rain
